In [1]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("GROQ_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser

llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=1)
chain = llm | StrOutputParser()

chain.invoke("Tell me joke about programming plzz do not regular joke")

'Why do programmers prefer dark mode?\n\nBecause light attracts bugs.'

## Structured Output

In [3]:
from typing import List
from pydantic import BaseModel, Field

class MobileReview(BaseModel):
    phone_model: str = Field(description="Name and model of the phone")
    rating: float = Field(description="Overall rating out of 5")
    pros: List[str] = Field(description="List of positive aspects")
    cons: List[str] = Field(description="List of negative aspects")
    summary: str = Field(description="Brief summary of the review")


review_text = """
Just got my hands on the new Galaxy S21 and wow, this thing is slick! The screen is gorgeous,
colors pop like crazy. Camera's insane too, especially at night - my Insta game's never been
stronger. Battery life's solid, lasts me all day no problem.

Not gonna lie though, it's pretty pricey. And what's with ditching the charger? C'mon Samsung.
Also, still getting used to the new button layout, keep hitting Bixby by mistake.

Overall, I'd say it's a solid 4 out of 5. Great phone, but a few annoying quirks keep it from
being perfect. If you're due for an upgrade, definitely worth checking out!
"""


structured_llm = llm.with_structured_output(MobileReview)
output = structured_llm.invoke(review_text)
output

MobileReview(phone_model='Galaxy S21', rating=4.0, pros=['gorgeous screen', 'colors pop', 'insane camera', 'solid battery life'], cons=['pricey', 'no charger', 'new button layout can be annoying'], summary='Great phone with a few annoying quirks, worth checking out for those due for an upgrade')

### Prompt Template 

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")

chain = prompt | llm | StrOutputParser()
chain.invoke({"topic: programmer"})

'Why do programmers prefer dark mode?\n\nBecause light attracts bugs.'

In [5]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

docx_loader = Docx2txtLoader("docs\GreenGrow Innovations_ Company History.docx")
documents = docx_loader.load()

print(len(documents))

splits = text_splitter.split_documents(documents)

print(f"Split the documents into {len(splits)} chunks.")

1
Split the documents into 2 chunks.


In [6]:
documents[0]

Document(metadata={'source': 'docs\\GreenGrow Innovations_ Company History.docx'}, page_content="GreenGrow Innovations was founded in 2010 by Sarah Chen and Michael Rodriguez, two agricultural engineers with a passion for sustainable farming. The company started in a small garage in Portland, Oregon, with a simple mission: to make farming more environmentally friendly and efficient.\n\n\n\nIn its early days, GreenGrow focused on developing smart irrigation systems that could significantly reduce water usage in agriculture. Their first product, the WaterWise Sensor, was launched in 2012 and quickly gained popularity among local farmers. This success allowed the company to expand its research and development efforts.\n\n\n\nBy 2015, GreenGrow had outgrown its garage origins and moved into a proper office and research facility in the outskirts of Portland. This move coincided with the development of their second major product, the SoilHealth Monitor, which used advanced sensors to analyze

In [7]:
splits[0].metadata

splits[0].page_content

'GreenGrow Innovations was founded in 2010 by Sarah Chen and Michael Rodriguez, two agricultural engineers with a passion for sustainable farming. The company started in a small garage in Portland, Oregon, with a simple mission: to make farming more environmentally friendly and efficient.\n\n\n\nIn its early days, GreenGrow focused on developing smart irrigation systems that could significantly reduce water usage in agriculture. Their first product, the WaterWise Sensor, was launched in 2012 and quickly gained popularity among local farmers. This success allowed the company to expand its research and development efforts.\n\n\n\nBy 2015, GreenGrow had outgrown its garage origins and moved into a proper office and research facility in the outskirts of Portland. This move coincided with the development of their second major product, the SoilHealth Monitor, which used advanced sensors to analyze soil composition and provide real-time recommendations for optimal crop growth.'

In [8]:
# Load documents from the folder

import os

def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents


folder_path = "docs"
documents = load_documents(folder_path)

print(f"Loaded: {len(documents)} documents from the folder.")
splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks")

Loaded: 5 documents from the folder.
Split the documents into 8 chunks


In [9]:
embeddings = OpenAIEmbeddings()

document_embeddings = embeddings.embed_documents([split.page_content for split in splits])

print(f"Created embeddings for {len(document_embeddings)} document chunks.")


Created embeddings for 8 document chunks.


In [10]:
from langchain_chroma import Chroma

embeddings_function = OpenAIEmbeddings()
vector_store = Chroma.from_documents(collection_name="my_collection", documents=splits, embedding=embeddings_function, persist_directory="./chroma_db")

In [11]:
query = "When was GreenGrow Innovations founded?"
search_results = vector_store.similarity_search(query, k=2)

for i, result in enumerate(search_results, 1):
    print(f"Result {i}:")
    print(f"Source: {result.metadata.get('source', 'Unknown')}")
    print(f"Content: {result.page_content}")

Result 1:
Source: docs\GreenGrow Innovations_ Company History.docx
Content: The company's breakthrough came in 2018 with the introduction of the EcoHarvest System, an integrated solution that combined smart irrigation, soil monitoring, and automated harvesting techniques. This system caught the attention of large-scale farmers across the United States, propelling GreenGrow to national prominence.



Today, GreenGrow Innovations employs over 200 people and has expanded its operations to include offices in California and Iowa. The company continues to focus on developing sustainable agricultural technologies, with ongoing projects in vertical farming, drought-resistant crop development, and AI-powered farm management systems.



Despite its growth, GreenGrow remains committed to its original mission of promoting sustainable farming practices. The company regularly partners with universities and research institutions to advance the field of agricultural technology and hosts annual confere

In [12]:
from langchain.schema.runnable import RunnablePassthrough

retriever = vector_store.as_retriever(search_kwargs={"k": 2})

template = """Answer the question base only on the following context:
{context}

Question: {question}
Answer: 
"""

def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

prompt = ChatPromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

question = "When was GreenGrow Innovations founded?"
response = rag_chain.invoke(question)
print(response)

The text does not explicitly state when GreenGrow Innovations was founded. However, it mentions that the company's breakthrough came in 2018 with the introduction of the EcoHarvest System, which propelled them to national prominence. This implies that the company was likely founded before 2018, but the exact date is not provided.


# Conversational RAG
### Handling Follow Up Question

In [13]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []
chat_history.extend([
    HumanMessage(content="question"),
    AIMessage(content=response)
])

In [14]:
chat_history

[HumanMessage(content='question', additional_kwargs={}, response_metadata={}),
 AIMessage(content="The text does not explicitly state when GreenGrow Innovations was founded. However, it mentions that the company's breakthrough came in 2018 with the introduction of the EcoHarvest System, which propelled them to national prominence. This implies that the company was likely founded before 2018, but the exact date is not provided.", additional_kwargs={}, response_metadata={})]

In [15]:
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

history_aware_retriever.invoke({"input": "Where it is headquartered?", "chat_history": chat_history})


[Document(id='0902d48a-84ef-48fa-af49-d25ca07a9679', metadata={'source': 'docs\\GreenGrow Innovations_ Company History.docx'}, page_content="The company's breakthrough came in 2018 with the introduction of the EcoHarvest System, an integrated solution that combined smart irrigation, soil monitoring, and automated harvesting techniques. This system caught the attention of large-scale farmers across the United States, propelling GreenGrow to national prominence.\n\n\n\nToday, GreenGrow Innovations employs over 200 people and has expanded its operations to include offices in California and Iowa. The company continues to focus on developing sustainable agricultural technologies, with ongoing projects in vertical farming, drought-resistant crop development, and AI-powered farm management systems.\n\n\n\nDespite its growth, GreenGrow remains committed to its original mission of promoting sustainable farming practices. The company regularly partners with universities and research institutions

In [16]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful Ai Assistant. Use the following context to answer the user's questions."),
        ("system", "Context: {context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}")
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [17]:
rag_chain.invoke({"input": "Where it is headquartered?", "chat_history": chat_history})

{'input': 'Where it is headquartered?',
 'chat_history': [HumanMessage(content='question', additional_kwargs={}, response_metadata={}),
  AIMessage(content="The text does not explicitly state when GreenGrow Innovations was founded. However, it mentions that the company's breakthrough came in 2018 with the introduction of the EcoHarvest System, which propelled them to national prominence. This implies that the company was likely founded before 2018, but the exact date is not provided.", additional_kwargs={}, response_metadata={})],
 'context': [Document(id='0902d48a-84ef-48fa-af49-d25ca07a9679', metadata={'source': 'docs\\GreenGrow Innovations_ Company History.docx'}, page_content="The company's breakthrough came in 2018 with the introduction of the EcoHarvest System, an integrated solution that combined smart irrigation, soil monitoring, and automated harvesting techniques. This system caught the attention of large-scale farmers across the United States, propelling GreenGrow to nationa

Building Multi User Chatbot

In [22]:
import sqlite3
from datetime import datetime

DB_NAME = "rag_app2.db"

def get_db_connection():
    conn = sqlite3.connect(DB_NAME)
    conn.row_factory = sqlite3.Row
    return conn

def create_application_logs():
    conn = get_db_connection()
    conn.execute('''CREATE TABLE IF NOT EXISTS application_logs
                    (id INTEGER PRIMARY KEY AUTOINCREMENT,
                     session_id TEXT,
                     user_query TEXT,
                     gpt_response TEXT,
                     model TEXT,
                     created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')
    conn.close()

def insert_application_logs(session_id, user_query, gpt_response, model):
    conn = get_db_connection()
    conn.execute('INSERT INTO application_logs (session_id, user_query, gpt_response, model) VALUES (?, ?, ?, ?)',
                 (session_id, user_query, gpt_response, model))
    conn.commit()
    conn.close()

def get_chat_history(session_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT user_query, gpt_response FROM application_logs WHERE session_id = ? ORDER BY created_at', (session_id,))
    messages = []
    for row in cursor.fetchall():
        messages.extend([
            {"role": "human", "content": row['user_query']},
            {"role": "ai", "content": row['gpt_response']}
        ])
    conn.close()
    return messages

# Initialize the database
create_application_logs()

In [23]:

import uuid
session_id = str(uuid.uuid4())
chat_history = get_chat_history(session_id)
print(chat_history)
question1 = "When was GreenGrow Innovations founded?"
answer1 = rag_chain.invoke({"input": question1, "chat_history":chat_history})['answer']
insert_application_logs(session_id, question1, answer1, "gpt-4-o-mini")
print(f"Human: {question1}")
print(f"AI: {answer1}\n")

[]
Human: When was GreenGrow Innovations founded?
AI: The provided context does not explicitly state when GreenGrow Innovations was founded. It only mentions that the company's breakthrough came in 2018 with the introduction of the EcoHarvest System, but it does not provide information about the founding date.



In [24]:
question2 = "Where it is headquartered?"
chat_history = get_chat_history(session_id)
print(chat_history)
answer2 = rag_chain.invoke({"input": question2, "chat_history":chat_history})['answer']
insert_application_logs(session_id, question2, answer2, "gpt-3.5-turbo")
print(f"Human: {question2}")
print(f"AI: {answer2}\n")

[{'role': 'human', 'content': 'When was GreenGrow Innovations founded?'}, {'role': 'ai', 'content': "The provided context does not explicitly state when GreenGrow Innovations was founded. It only mentions that the company's breakthrough came in 2018 with the introduction of the EcoHarvest System, but it does not provide information about the founding date."}]
Human: Where it is headquartered?
AI: The provided context does not explicitly state the headquarters of GreenGrow Innovations. It only mentions that the company has expanded its operations to include offices in California and Iowa, but it does not specify which location serves as the headquarters.



In [25]:
session_id = str(uuid.uuid4())
question = "What is GreenGrow"
chat_history = get_chat_history(session_id)
print(chat_history)
answer = rag_chain.invoke({"input": question, "chat_history":chat_history})['answer']
insert_application_logs(session_id, question, answer, "gpt-3.5-turbo")
print(f"Human: {question}")
print(f"AI: {answer}\n")

[]
Human: What is GreenGrow
AI: GreenGrow Innovations is a company that was founded in 2010 by two agricultural engineers, Sarah Chen and Michael Rodriguez. The company's mission is to make farming more environmentally friendly and efficient. They develop innovative products, such as smart irrigation systems and soil health monitoring tools, to help achieve this goal.

